# 웹채팅 프로그램 만들기

* 난이도 : ★★★☆☆☆☆☆☆☆
* 필요라이브러리: Flask, flask_socketio


![chat_5.jpg](images/chat_5.jpg)



* 채팅 강좌의 마지막으로 웹기반 채팅 프로그램을 만들어 봅니다.
* 웹채팅 프로그램은 이전 강좌에서 다룬 Flask 기반으로 작성합니다.
* 소켓 통신을 하기 위해 flask_socketio 라이브러리를 사용합니다.
> pip install flask_socketio

## 웹 클라이언트

* HTML 페이지로 구성되어 채팅 클라언트 역할 구현
* HTML 내용은 채팅 목록을 표시할 div 와 입력 박스, 전송 버튼으로 단순한 요소로 구성합니다.
* 소켓 통신을 사용하기 위해 자바스크립트 socket.io 라이브러리를 사용합니다.
> &#60;script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/1.7.4/socket.io.min.js"&#62;&#60;/script>
* HTML 에서 보다 편리한 요소 접근을 위해 jquery 라이브러리를 사용합니다.
> &#60;script src="https://code.jquery.com/jquery-2.2.4.min.js"&#62;&#60;/script>

In [ ]:
%%html
<div class="messages"></div>
<form>
    <input type="text" class="nickname">
    <input type="text" class="message">
    <input type="submit" value="전송하기">
</form>

<script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/1.7.4/socket.io.min.js"></script>
<script src="https://code.jquery.com/jquery-2.2.4.min.js"></script>
<script type="text/javascript">

    // io 객체는 socket.io.min.js 안에 선언된 소켓 객체 입니다.
    // 현재 접속된 주소:포트로 접속하여 socket 변수에 저장합니다.
    var socket = io.connect("http://" + document.domain + ":" + location.port);
    
    // socket에 connect 이벤트가 발생하면
    socket.on("connect", function() {
        // 서버로 connected 이벤트를 발생시킵니다.
        socket.emit("connected");

        // 전송하기를 클릭하거나 엔터를 입력하여 form 이 전송될때 수행하는 함수
        var form = $("form").on("submit", function(e) {
            // form 의 submit 전송을 취소합니다.
            // 이는 form이 정상적으로 submit 되면 페이지 리로드가 일어나는데
            // 페이지 리로드가 되면 기존 채팅 목록이 사라지기 때문에
            // 폼의 submit 이벤트를 강제로 취소시키는 겁니다.
            e.preventDefault();

            // input 의 클래스명이 nickname 인 요소의 값을 nickname 변수에 저장
            var nickname = $("input.nickname").val();
            // input 의 클래스명이 message 인 요소의 값을 message 변수에 저장
            var message = $("input.message").val();

            // 클라이언트(웹 HTML) 쪽으로 chat 이벤트를 발생시킵니다.
            socket.emit("chat", {
                nickname: nickname,
                message: message
                });
            $("input.message").val("").focus();
        })
    });

    // 서버에서 response 이벤트가 발생하면 (화면에 표기되는 채팅내용, 신규유저 알림이 response 이벤트로 옴)
    socket.on("response", function(msg) {
        // div 클래스명 messages 에 태그를 추가 합니다.
        $("div.messages").append("<div><b style='color:blue'>" + msg.nickname + "</b> " + msg.message + "</div>");
    });
</script>

## 서버 구현

* 기본적으로 flask 기반이지만 flask 에서 소켓을 사용하기 위해 flask_socketio 라이브러리를 추가로 사용합니다. 
* 서버와 웹클라이언트는 socket 기반으로 동작하며 event 를 발생시키고 받음으로서 통신을 주고 받습니다.

In [ ]:
# 플라스크 라이브러리
from flask import Flask
from flask import render_template

# 플라스크 소켓 통신을 위한 라이브러리
from flask_socketio import SocketIO

# flask 인스턴스 생성
app = Flask(__name__)

# Socket을 사용하기 위한 SocketIO 에 인자로 플라스크 app 변수를 넘겨 새로운 socketio 변수에 저장합니다.
socketio = SocketIO(app)

# 접속 주소는 / 입니다.
@app.route("/")
def index():
    # templates 폴더의 index.html 파일을 렌더링 해서 출력합니다.
    return render_template("index.html")

# 클라이언트에서 소켓으로 chat 이벤트가 발생하면
@socketio.on("chat")
def event_handler(json):
    # 인자로 넘어온 json 은 기본적으로 latin1 로 인코딩 되어있습니다.
    # 그래서 정상적인 한글을 사용하기 위해선 utf-8로 디코딩 해주어야 합니다.
    json["nickname"] = json["nickname"].encode("latin1").decode("utf-8")
    json["message"] = json["message"].encode("latin1").decode("utf-8")
    
    # 클라이언트에게 response 이벤트를 발생시킵니다.
    # 전송받은 닉네임과 채팅 메세지를 각각 nickname, message를 키값으로 dict형태로 전송합니다.
    socketio.emit("response", {"nickname":json["nickname"], "message": json["message"]})

# 클라이언트에서 소켓으로 connectd 이벤트가 발생하면
@socketio.on("connected")
def connect_handler():
    # 클라이언트에게 response 이벤트를 발생시킵니다.
    # response 이벤트는 nickname 과 message를 키로 받기 때문에 키 값은 고정으로 두고
    # message 값만 '새로운 유저 입장' 이라고 알려줍니다.
    socketio.emit("response", {"nickname":"", "message": "새로운 유저 입장"})

# py 파일 시작 엔트리 포인트
if __name__ == "__main__":
    # socketio 를 run 합니다.
    # 기본적으로 flask 인스턴스를 run 하는것과 같습니다.
    socketio.run(app, host="0.0.0.0", port=14000, debug=True)